In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *


In [0]:
!pip install kaggle


**UPLOAD KAGGLE json FILE**

In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


**Create path where you should download data**

In [0]:
path = Path('./sample_data/planet')
path.mkdir(parents=True, exist_ok=True)

**Download data**

In [10]:
!kaggle competitions download -c planet-understanding-the-amazon-from-space -p {path}

  0% 0.00/151k [00:00<?, ?B/s]
100% 151k/151k [00:00<00:00, 47.8MB/s]
  0% 0.00/600k [00:00<?, ?B/s]
100% 600k/600k [00:00<00:00, 83.4MB/s]
  0% 0.00/148k [00:00<?, ?B/s]
100% 148k/148k [00:00<00:00, 47.0MB/s]
  0% 0.00/1.01M [00:00<?, ?B/s]
100% 1.01M/1.01M [00:00<00:00, 68.4MB/s]
  0% 0.00/1.52M [00:00<?, ?B/s]
100% 1.52M/1.52M [00:00<00:00, 104MB/s]


**Unzip the downloaded files**

In [0]:
! unzip -q -n {path}/train_v2.csv.zip -d {path}

In [12]:
!ls
# Note that there won't be any train-jpg

sample_data


**That above code won't download train-jpg zip file due to some updates in this dataset, so use idm to capture the url of that file and do below steps**

**MOVE TO IDM DOWNLOADED, SKIP STEPS IN BETWEEN**

In [0]:
Url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/6322/868312/upload/test-jpg.tar.7z?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1578943958&Signature=HpO8cpIHdSl0OTdpp7IrLdK8gR0AM06ApM0No8MzdBowtfSZkXJwTlTHTk%2B%2B0Fwm7ZRE5LTlDC2IegYJyz%2BmFba2bFRJUVyaZsiH3%2Bal%2BqJQkuT%2FqUStsLpkq6WtkGk6r5p7USjEq8Z2sBISnLPl2kYc21VzSsZh7EJ44MqCGjWFw33k9nsSFM9RpFj%2BP3TfDgCs%2B7z4SzDyWzE5TjuEc7nyl2FmWdVoy7zD9zWm8UOXHXLe%2FThkem9r%2FtQXVa6I7r8imu8ZQDAtCeIMDR38icLU0cuONHTpd4Nt5duy7G%2BnaON5JJ6oUwcOKAviVeDLiuMxqauurTKVs3DhGZxzoA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest-jpg.tar.7z'

download_data(Url, fname = 'f')

**Unzip the downloaded file and save with name**

In [0]:
! unzip -q -n {path3}/train-jpg.tgz -d {path3}

In [33]:
untar_data( Url,fname='h.tar.7z',dest='kl')

ReadError: ignored

**IDM DOWNLOADED**

**VIEW THE CLASSES**

In [43]:
df = pd.read_csv(path/'train_v2.csv')
df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


**Transforms**

In [0]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)


**src**

In [0]:
np.random.seed(42)
src = (ImageList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=' '))

In [0]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

show_batch still works, and show us the different labels separated by ;.



In [0]:
data.show_batch(rows=3, figsize=(12,9))

**LEARNER**

In [0]:
arch = models.resnet50


In [0]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(data, arch, metrics=[acc_02, f_score])

**LR**

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

**FIT once CYCLE**

In [0]:
lr = 0.01

In [0]:
learn.fit_one_cycle(5, slice(lr))

**SAVE**

In [0]:
learn.save('stage-1-rn50')


**TUNING**

In [0]:
learn.unfreeze()


In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [0]:
learn.save('stage-2-rn50')

In [0]:
data = (src.transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

In [0]:
learn.freeze()


In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
lr=1e-2/2

In [0]:
learn.fit_one_cycle(5, slice(lr))

In [0]:
learn.save('stage-1-256-rn50')


In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [0]:
learn.recorder.plot_losses()

In [0]:
learn.save('stage-2-256-rn50')

In [0]:
learn.export()